# Sentiment Analysis on IMDB Reviews using LSTM


### Steps
<ol type="1">
    <li>Load the dataset</li>
    <li>Clean Dataset</li>
    <li>Encode Sentiments</li>
    <li>Split Dataset</li>
    <li>Tokenize and Pad/Truncate Reviews</li>
    <li>Build Architecture/Model</li>
    <li>Train and Test</li>
</ol>




In [56]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re
from tensorflow.keras import Input

In [35]:
data = pd.read_csv('IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [36]:
english_stops = set(stopwords.words('english'))

In [37]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case

    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


C:\Users\nitin\AppData\Local\Temp\ipykernel_22972\4067306172.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_data = y_data.replace('negative', 0)


In [38]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
44362    [my, rd, year, french, classes, always, enjoye...
40054    [there, seems, overwhelming, response, movie, ...
31240    [some, spoilers, included, although, many, com...
4797     [greetings, darkness, insight, mind, motivatio...
22284    [falsely, accused, skirt, chasing, chums, john...
                               ...                        
37683    [when, i, think, period, film, i, think, i, wa...
39734    [i, seen, movie, recently, although, appeared,...
10988    [i, work, blockbuster, store, every, week, mov...
21243    [i, set, high, expectations, movie, left, plea...
11424    [a, quiet, sweet, beutifully, nostalgic, movie...
Name: review, Length: 40000, dtype: object 

49131    [i, guess, everyone, make, comeback, point, an...
43138    [this, movie, horrendous, it, absolutely, noth...
22498    [the, events, september, need, extra, human, i...
2185     [peter, lorre, turns, one, finest, performance...
31983    [blademaster, definitely, memorable, entry, sw...
 

In [39]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [40]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[ 213 3863  192 ...    0    0    0]
 [  50   94 4340 ...   45  571 1390]
 [ 390  967 1813 ...  151   12   96]
 ...
 [   1   75 2637 ...    0    0    0]
 [   1  173  204 ...    0    0    0]
 [  39 1940  992 ...    0    0    0]] 

Encoded X Test
 [[    1   383   193 ...     0     0     0]
 [    8     3  3496 ...     0     0     0]
 [    2   619  5722 ...     0     0     0]
 ...
 [  333    78 78756 ...     0     0     0]
 [ 1404     3    12 ...     0     0     0]
 [   16    12  4778 ...     0     0     0]] 

Maximum review length:  130


In [57]:

EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential([
    Input(shape=(max_length,)),                      # 👈 define input here
    Embedding(total_words, EMBED_DIM),               # no input_length needed
    LSTM(LSTM_OUT),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 130, 32)        │     2,960,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 64)             │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,985,089 (11.39 MB)

 Trainable params: 2,985,089 (11.39 MB)

 Non-trainable params: 0 (0.00 B)

In [58]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [59]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
312/313 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.5419 - loss: 0.6846
Epoch 1: accuracy improved from -inf to 0.58495, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 60ms/step - accuracy: 0.5422 - loss: 0.6845
Epoch 2/5
312/313 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.6362 - loss: 0.6405
Epoch 2: accuracy improved from 0.58495 to 0.63320, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 61ms/step - accuracy: 0.6362 - loss: 0.6405
Epoch 3/5
312/313 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.7096 - loss: 0.5838
Epoch 3: accuracy improved from 0.63320 to 0.73530, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 60ms/step - accuracy: 0.7097 - loss: 0.5836
Epoch 4/5
312/313 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.8062 - loss: 0.4706
Epoch 4: accuracy improved from 0.73530 to 0.81327, saving model to models/LSTM.h5


313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 59ms/step - accuracy: 0.8062 - loss: 0.4705
Epoch 5/5
312/313 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.8429 - loss: 0.3669
Epoch 5: accuracy did not improve from 0.81327
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 60ms/step - accuracy: 0.8425 - loss: 0.3674


In [60]:
# Get probabilities
y_prob = model.predict(x_test, batch_size=128)

# Convert to class labels (0 or 1)
y_pred = (y_prob > 0.5).astype("int32").flatten()

true = np.sum(y_test.values == y_pred)
wrong = len(y_pred) - true
accuracy = true / len(y_pred) * 100

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(wrong))
print('Accuracy: {:.2f}%'.format(accuracy))

79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step
Correct Prediction: 8201
Wrong Prediction: 1799
Accuracy: 82.01%


In [61]:
loaded_model = load_model('models/LSTM.h5')

In [52]:
review = str(input('Movie Review: '))

In [47]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  
Filtered:  ['']


In [48]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [49]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
[[0.18137655]]


In [50]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

negative
